In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [79]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [81]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['saved_library_scope']
redirect_uri = spotify_creds['saved_library_redirect_url']

In [83]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [84]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [85]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [86]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [87]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [88]:
df = get_saved_library()

In [116]:
songs = sp.current_user_saved_tracks(offset=0)
songs['items'][0]['track']['external_urls']

{'spotify': 'https://open.spotify.com/track/2XKsHHNCtKqk9cF35TRFyC'}

In [120]:
str1 = songs['items'][0]['track']['external_urls']['spotify']

In [123]:
str1.split('/')[-1]

'2XKsHHNCtKqk9cF35TRFyC'

In [33]:
scope = 'user-top-read'
redirect_uri = 'http://localhost:8885/callback'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [34]:
sp = spotipy.Spotify(auth=token)

In [137]:
list(df['song_title'])

['Do It To It - Tiësto Remix',
 'Freaky Deaky',
 'Nightmare',
 "If I Ain't Got You - Acoustic",
 "If I Ain't Got You",
 'Lost At Sea (Illa Illa 2)',
 "When I'm Gone (with Katy Perry) [VIP Mix]",
 'Control',
 'Moments',
 'redruM',
 'Walk Me Home',
 '半島鐵盒',
 'Ghost Of My Past',
 'Empty Love',
 'Don’t Wake Me Up',
 '失眠飛行',
 'Into The Fire',
 'Afterglow',
 'Tired - Steerner & Tobu Remix',
 'Angels - Radio Edit',
 '裂心',
 'Coming Home',
 '痴心絕對',
 '後來',
 "Squid Game (Let's Play)",
 'WITHOUT YOU',
 'Just Look Up (From Don’t Look Up)',
 'Close Your Eyes - VIP Mix',
 'Most People - R3HAB VIP Remix',
 'Hope',
 'Christmas Got Colder',
 'Lalala',
 'Love Is Gone (Tiësto Remix)',
 'God Mode',
 'Frozen Fire',
 'God is A DJ',
 'Kiss Me More (feat. SZA)',
 'Love Back',
 'Lay It Down',
 'Last Christmas',
 'California (feat. Warren Hue)',
 'Maze',
 'Unstable (feat. The Kid LAROI)',
 'After You (feat. Calle Lehmann)',
 'Ride With Me (feat. Kid Ink) - Blasterjaxx & Tungevaag Remix',
 'Unchained',
 'Bassman'